# FFWIZARD.COM --- Draft Rankings Generator

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from io import StringIO
from sklearn import preprocessing
from pandas import ExcelWriter 
from datetime import datetime

### Pull in ADP Data

In [2]:
def pull_adp(ppr, teams):
    
    # pull adp url with ppr and teams params
    url = 'https://fantasyfootballcalculator.com/adp/csv/'+ ppr +'.csv?teams='+ teams +'&position=all'
    page_raw = requests.get(url)
    page_txt = str(page_raw.text)[:]

    # pull in text starting where data is...
    bye_char = page_txt.find("Bye") + 4
    last_char = page_txt.find("</pre>")

    page_data = page_txt[bye_char:last_char]

    # set up list and initialize dataframe
    split_txt = page_data.splitlines()
    df = pd.DataFrame(split_txt[0].split(",")).T

    # iterate through lists and form dataframe
    num_players = len(split_txt) - 1
    for i in range(1, num_players):
        new_row = pd.DataFrame(split_txt[i].split(",")).T
        df = pd.concat([df, new_row], axis=0)

    # format dataframe and include relevant info
    df = df.reset_index().iloc[:, [2, 3, 4, 5, 10]]
    df.columns = ["Rank", "Name", "Pos", "Team", "Bye"]
    
    print(url)
    
    return(df)

### Remove Free Agents

In [3]:
def modify_fa(df, put_fa_on_team):
    
    for i in range(0, len(df)):
        for j in range(0, len(put_fa_on_team)):
            if df.loc[i, "Name"] == put_fa_on_team.iloc[j, 0]:
                df.loc[i, "Team"] = put_fa_on_team.iloc[j, 1]
    
    df_fa_idx = df[df.Team == "FA"].index
    df_no_fa = df.drop(df_fa_idx).reset_index(drop=True)

    return df_no_fa

In [4]:
# df_no_fa = modify_fa(df, put_fa_on_team)
# df_no_fa

### Remove Duplicates

In [5]:
def remove_duplicates(df):
    
    drop_duplicates_idx = []
    
    for i in range(1, len(df)):
        if df.loc[i, "Name"] == df.loc[(i-1), "Name"]:
            drop_duplicates_idx.append(i)

    new_df = df.drop(drop_duplicates_idx).reset_index(drop=True)

    return new_df

In [6]:
# df = remove_duplicates(df)
# df

### Drop Injured Players

In [7]:
def drop_injured_players(df_ffc, injured_players):

    rows_drop = []

    for i in range(0, len(df_ffc)):
        for j in range(0, len(injured_players)):

            if (df_ffc.loc[i, 'Name'] == injured_players[j]):

                rows_drop.append(i)

    new_df = df_ffc.drop(rows_drop).reset_index()
    
    return new_df

In [8]:
# df = drop_injured_players(df, injured_players)
# df

### Load NFL Schedule

In [9]:
# load nfl schedule data (manually entered)
def load_schedule():
    nfl_sched = pd.read_csv('2021_NFL_Schedule.csv')
    return nfl_sched

In [10]:
# nfl_sched = load_schedule()
# nfl_sched

### Defense Strength Data

In [11]:
def calc_defense_strength(bye_adjustor):

    # load nfl defense strength data (manually entered)
    def_strength_data = pd.read_csv('2021_Defense_Strength.csv')

    #remove 'source' row
    def_strength = def_strength_data.iloc[:-1, :] # take off footnote row of text

    # make weights based on quality of rankings
    def_rank_weights = pd.Series([0.20, 0.20, 0.20, 0.10, 0.10, 0.10, 0.10])
    
    # create weighted defense rankings array based on weights
    num_teams = len(def_strength)
    weighted_def_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_def_rank[i] = def_rank_weights[0] * float(def_strength.iloc[i, 1]) + def_rank_weights[1] * float(def_strength.iloc[i, 2]) + def_rank_weights[2] * float(def_strength.iloc[i, 3]) + def_rank_weights[3] * float(def_strength.iloc[i, 4])
    weighted_def_rank_df = pd.Series(weighted_def_rank)

    # scale weighted defense rankings to have mean of 0 and std dev of 1
    def_rank_scaled = pd.Series(preprocessing.scale(weighted_def_rank_df))

    # create table with team and score info
    defense_score_no_bye = pd.concat([def_strength.loc[:, "Team"], weighted_def_rank_df, def_rank_scaled], axis=1)
    col_names = ["Team", "Def_Score", "Def_Score_Scaled"]
    defense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    defense_score = pd.concat([defense_score_no_bye, bye_array])

    defense_score = defense_score.reset_index(drop=True)
    
    return(defense_score)

In [12]:
# defense_score = calc_defense_strength(bye_adjustor)
# defense_score

### Offense Strength Data

In [13]:
def calc_offense_strength(bye_adjustor):

    # load nfl offense strength data (manually entered)
    off_strength_data = pd.read_csv('2021_Offense_Strength.csv')

    # remove 'source' row
    off_strength = off_strength_data.iloc[:-1, :]   # take off footnote row of text

    # make weights based on quality of rankings
    off_rank_weights = pd.Series([0.30, 0.30, 0.10, 0.10, 0.15, 0.05])
    
    # create weighted offense rankings array based on weights
    num_teams = len(off_strength)
    weighted_off_rank = np.empty(num_teams, dtype=float)
    
    #### REDO THIS FOR LOOP...maybe just feed weighted defense rankings....maybe allow user to choose which weighting system they want!
    for i in range(0, num_teams):
        weighted_off_rank[i] = off_rank_weights[0] * float(off_strength.iloc[i, 1]) + off_rank_weights[1] * float(off_strength.iloc[i, 2]) + off_rank_weights[2] * float(off_strength.iloc[i, 3]) + off_rank_weights[3] * float(off_strength.iloc[i, 4])  + off_rank_weights[4] * float(off_strength.iloc[i, 5])
    weighted_off_rank_df = pd.Series(weighted_off_rank)
    #### REDO THIS FOR LOOP...
    
    # scale weighted offense rankings to have mean of 0 and std dev of 1
    off_rank_scaled = pd.Series(preprocessing.scale(weighted_off_rank_df))

    # create table with team and score info
    offense_score_no_bye = pd.concat([off_strength.loc[:, "Team"], weighted_off_rank_df, off_rank_scaled], axis=1)
    col_names = ["Team", "Off_Score", "Off_Score_Scaled"]
    offense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    offense_score = pd.concat([offense_score_no_bye, bye_array])

    offense_score = offense_score.reset_index(drop=True)
    
    return offense_score

In [14]:
# offense_score = calc_offense_strength(bye_adjustor)
# offense_score

### Add Schedule to ADP Ranks

In [15]:
def sched_per_player(df, nfl_sched):

    # create blank table to store opponent names
    opponents = pd.DataFrame(np.zeros((len(df),18)), columns = ["Opp_1", "Opp_2", "Opp_3", "Opp_4", "Opp_5", "Opp_6", "Opp_7", "Opp_8", "Opp_9", "Opp_10", "Opp_11", "Opp_12", "Opp_13", "Opp_14", "Opp_15", "Opp_16", "Opp_17", "Opp_18"])

    # iterate through each player and every week
    for i in range(0, len(df)):
        for j in range(0, len(nfl_sched)):

            # set weekly schedule for each player based on team
            if (df.loc[i, "Team"] == nfl_sched.loc[j, "Team"]):
                
                # create scehdules for each ranked player
                for k in range(0,18):
                    opponents.iloc[i, k] = nfl_sched.iloc[j, (k+1)]
        
    return opponents

In [16]:
# opponents = sched_per_player(df, nfl_sched)

### Create Matchup Strength Table by Week

In [17]:
def weekly_matchup_strength(df, opponents_df, offense_score, defense_score):

    df_pos = df.loc[:, 'Pos'].values

    df_def_idx = (df_pos == "DEF")
    # df_def = df.iloc[df_def_idx,:]
    opponents_df_def = opponents_df.iloc[df_def_idx,:]

    df_off_idx = (df_pos != "DEF")
    # df_off = df.iloc[df_off_idx,:]
    opponents_df_off = opponents_df.iloc[df_off_idx,:]

    opp_df_off_strength = opponents_df_off.copy()

    for i in range(0, len(defense_score)):
        team = defense_score.loc[i,'Team']
        def_score = defense_score.loc[i,'Def_Score_Scaled']
        opp_df_off_strength.replace(team, def_score, inplace=True)



    opp_df_def_strength = opponents_df_def.copy()

    for j in range(0, len(offense_score)):
        team = offense_score.loc[j,'Team']
        off_score = offense_score.loc[j,'Off_Score_Scaled']
        opp_df_def_strength.replace(team, off_score, inplace=True)


    str_scores_unwgt = pd.concat((opp_df_off_strength, opp_df_def_strength),axis=0).sort_index()    
    
    str_scores_unwgt.columns = ["Str_1", "Str_2", "Str_3", "Str_4", "Str_5", "Str_6", "Str_7", "Str_8", "Str_9", "Str_10", "Str_11", "Str_12", "Str_13", "Str_14", "Str_15", "Str_16", "Str_17", "Str_18"]

    return str_scores_unwgt

In [18]:
# str_scores_unwgt = weekly_matchup_strength(df, opponents, offense_score, defense_score)
# str_scores_unwgt

### Make Week-by-Week Weightings Array (Higher for Early Weeks and Playoffs)

In [19]:
def week_weight_table(first_playoff_week, last_playoff_week, week_weights_exponent, playoff_weight, week_weights_thru_wk14_qb_wr_k_def, week_weights_thru_wk14_rb_te):
    
    # tune level of "depreciation" curve applied to players, i.e. week weight inputs accounting for injury/trade.
    # exponent = 0: turn OFF week 1-13 weights
    # exponent = 1: turn ON 100% of week weights
    
    thru_14_qb_wr_k_def = list(np.power(week_weights_thru_wk14_qb_wr_k_def, week_weights_exponent))
    thru_14_rb_te = list(np.power(week_weights_thru_wk14_rb_te, week_weights_exponent))
    
    # make blank list to store possible playoff weeks, 14 to 18
    playoff_weeks_qb_wr_k_def = []
    playoff_weeks_rb_te = []

    # week 15
    if (first_playoff_week == 15):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(week_weights_thru_wk14_qb_wr_k_def[13])
        playoff_weeks_rb_te.append(week_weights_thru_wk14_rb_te[13])

    # week 16
    if (first_playoff_week >= 15):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(week_weights_thru_wk13_qb_wr_k_def[12])
        playoff_weeks_rb_te.append(week_weights_thru_wk13_rb_te[12])

    # week 17
    if (last_playoff_week >= 17):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(0.0)
        playoff_weeks_rb_te.append(0.0)

    # week 18
    if (last_playoff_week == 18):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(0.0)
        playoff_weeks_rb_te.append(0.0)

    # combine weeks 1 to 14 with playoff weeks
    ### ADD one ZERO at end for weight of week 18
    week_weights_qb_wr_k_def = thru_14_qb_wr_k_def + playoff_weeks_qb_wr_k_def
    week_weights_rb_te = thru_14_rb_te + playoff_weeks_rb_te
    
    print(week_weights_qb_wr_k_def)
    
    return week_weights_qb_wr_k_def, week_weights_rb_te

In [20]:
# week_weights_qb_wr_k_def, week_weights_rb_te = week_weight_table(first_playoff_week, last_playoff_week, week_weights_exponent, playoff_weight, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te)
# week_weights_qb_wr_k_de

### Apply Week-by-Week Weights to Player Weekly Matchups

In [21]:
def weight_str_scores(df, str_scores_unwgt, week_weights_qb_wr_k_def, week_weights_rb_te, first_playoff_week, last_playoff_week, playoff_weight):

    # pull in adp ranks as floats
    df_rank = df.loc[:, "Rank"]
    df_rank = map(float, df_rank)
    df_rank_array = np.array(df_rank)
    
    # make copy of weekly matchup strengths for each player
    str_scores_wgt = str_scores_unwgt.copy()
    
    # iterate through each row and multiply matchup strengths by weekly weights
    for i in range(0, len(str_scores_wgt)):
        
        # if player position is RB or TE, apply weight curve reflecting greater chance of injury
        if (df.loc[i, "Pos"] == "RB") or (df.loc[i, "Pos"] == "TE"):
            wgt_str_scores = np.multiply(str_scores_unwgt.iloc[i, :], week_weights_rb_te)
            str_scores_wgt.iloc[i, :] = wgt_str_scores
        
        # otherwise apply different curve
        else:
            wgt_str_scores = np.multiply(str_scores_unwgt.iloc[i, :], week_weights_qb_wr_k_def)
            str_scores_wgt.iloc[i, :] = wgt_str_scores
        
    for j in range(0, len(str_scores_wgt)):
        for k in range(first_playoff_week-1, last_playoff_week):
            str_scores_wgt.iloc[j, k] = np.multiply(playoff_weight,str_scores_unwgt.iloc[j, k])

    # put weighted matchup strengths into organized dataframe
    str_scores_wgt.columns = ["Wgt_1", "Wgt_2", "Wgt_3", "Wgt_4", "Wgt_5", "Wgt_6", "Wgt_7", "Wgt_8", "Wgt_9", "Wgt_10", "Wgt_11", "Wgt_12", "Wgt_13", "Wgt_14", "Wgt_15", "Wgt_16", "Wgt_17", "Wgt_18"]
    
    return str_scores_wgt

In [22]:
# str_scores_wgt = weight_str_scores(df, str_scores_unwgt, week_weights_qb_wr_k_def, week_weights_rb_te, first_playoff_week, last_playoff_week)


### Overall Strength Rating Per Player

In [23]:
def player_weighted_sched_str(str_scores_wgt):

    # set up and populate strength array (weighted)
    str_scores_sum_wgt = np.zeros(len(str_scores_wgt))
    for i in range(0, len(str_scores_wgt)):
        str_scores_sum_wgt[i] = str_scores_wgt.iloc[i, :].sum()

    # scale sos values (mean = 0, var = 1)
    str_scores_sum_scaled = pd.Series(preprocessing.scale(str_scores_sum_wgt))
    
    return str_scores_sum_scaled

### Compute Strength of Schedule Score

In [24]:
def ffwiz_score_scaled(mccaffrey_penalty, barkley_penalty, najee_penalty, df, str_scores_sum_scaled, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster):

    df_rank = np.zeros((len(df)))
    
    # pull in adp ranks and SoS scores
    for i in range(0,len(df)):
        df_rank[i] = float(df.loc[i, "Rank"])
    

    for i in range(0, len(df)):
        
        if df.Name[i] == "Christian McCaffrey":
            mccaffrey_row = i

        if df.Name[i] == "Saquon Barkley":
            barkley_row = i
            
        if df.Name[i] == "Najee Harris":
            najee_row = i

    df_rank[mccaffrey_row] = df_rank[mccaffrey_row] + mccaffrey_penalty

    df_rank[barkley_row] = df_rank[barkley_row] + barkley_penalty
            
    df_rank[najee_row] = df_rank[najee_row] + najee_penalty
            
            
    df_rank_array = df_rank.copy()
    
    # initialize adjusted rankings
    df_rank_adj = df_rank_array.copy()

    
    # iterate through the players and add penalty to DEF and K
    for i in range(0, len(df)):
        if df.loc[i, "Pos"] == "DEF":
            df_rank_adj[i] = df_rank_array[i] + rank_adder + def_rank_penalty
        elif df.loc[i, "Pos"] == "PK":
            df_rank_adj[i] = df_rank_array[i] + rank_adder + k_rank_penalty
        elif df.loc[i, "Pos"] == "WR":
            df_rank_adj[i] = df_rank_array[i] + rank_adder - wr_rank_booster        
        else:
            df_rank_adj[i] = df_rank_array[i] + rank_adder

    # adjust adp ranks and SoS scores (to move players up/down appropriate slots)
    str_scores_sum_scaled_adj = matchup_strength_scaler - str_scores_sum_scaled

    # multiply adjusted adp ranks and SoS scores
    df_rank_adj_scaled = np.multiply(df_rank_adj, str_scores_sum_scaled_adj)
    ffwiz_score = np.asarray(df_rank_adj_scaled)
    
    return ffwiz_score

In [25]:
# ffwiz_score = ffwiz_score_scaled(df, str_scores_sum_scaled, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster)
# ffwiz_score

### Overall FFWIZ Rankings

In [26]:
def ffwiz_overall_rankings(df, ffwiz_score):

    # sort strength of schedule scores and put into dataframe
    ffwiz_score_df = pd.DataFrame(ffwiz_score, columns=["FFWIZ_Score"])
    ffwiz_score_sort = ffwiz_score_df.sort_values(by='FFWIZ_Score')
    ffwiz_sort_idx = ffwiz_score_sort.index.values + 1
    ffwiz_sort_df = pd.DataFrame(ffwiz_sort_idx, columns=["FFWIZ"])

    # get ffwiz overall rankings using above dataframe
    ff_sort = ffwiz_sort_df.sort_values(by='FFWIZ')
    ff_sort_idx = ff_sort.index.values + 1
    ff_sort_df = pd.DataFrame(ff_sort_idx, columns=["FFWIZ"])

    # make dataframe for adp rankings
    adp_values = np.array(range(1, len(ffwiz_sort_idx)+1))
    adp_rank_df = pd.DataFrame(adp_values, columns=["ADP"])


    # calculate difference in ffwiz vs. adp rankings
    diff = pd.DataFrame(np.asarray(adp_rank_df) - np.asarray(ff_sort_df), columns=["DIFF"])

    # make master dataframe and sort
    ffwiz_df = pd.concat([ff_sort_df, adp_rank_df, diff, df.iloc[:, 1:]], axis=1)
    ffwiz_df_overall = ffwiz_df.sort_values(by='FFWIZ').reset_index(drop=True)

    print(ffwiz_df_overall.iloc[:40, :])
    
    return ffwiz_df_overall

In [27]:
# ffwiz_df_overall = ffwiz_overall_rankings(df, ffwiz_score)
# ffwiz_df_overall

### Positional FFWIZ Rankings

In [28]:
def ffwiz_df_positional(ffwiz_df_overall):

    # FFWIZ
    # sort by position and add FFWIZ and ADP positional rankings
    ffwiz_df_no_pos_rank = ffwiz_df_overall.sort_values(by=['Pos', 'FFWIZ']).reset_index(drop=True)

    # add positional ranks to FFWIZ rankings
    ffwiz_pos_rank_list = [1] #initialize pos list
    ffwiz_rank_counter = 1 #initialize positional rank to 1

    # populate positional rank list
    for k in range(1, len(ffwiz_df_overall)):

        # reset positional rank value if 
        if (ffwiz_df_no_pos_rank.loc[k, 'Pos'] == ffwiz_df_no_pos_rank.loc[k-1, 'Pos']):
            ffwiz_rank_counter = ffwiz_rank_counter + 1
        else:
            ffwiz_rank_counter = 1

        ffwiz_pos_rank_list.append(ffwiz_rank_counter)

    ffwiz_df_pos_rank = pd.DataFrame(ffwiz_pos_rank_list, columns=["FFWIZ_Pos_Rank"]).reset_index(drop=True)

    
    # sort by position and add and ADP positional rankings
    adp_df_no_pos_rank = ffwiz_df_overall.sort_values(by=['Pos', 'ADP']).reset_index(drop=True)
    adp_pos_rank_table = pd.concat([adp_df_no_pos_rank, ffwiz_df_pos_rank], axis=1)
    adp_pos_sort_ffwiz = adp_pos_rank_table.sort_values(by=['Pos', 'FFWIZ']).reset_index(drop=True)
    adp_pos_rank_array = np.asarray(adp_pos_sort_ffwiz.loc[:, 'FFWIZ_Pos_Rank'])
    adp_pos_rank = pd.DataFrame(adp_pos_rank_array, columns=["ADP_Pos_Rank"])

    # DIFF & CONCATENATE
    ffwiz_pos_rank_array = np.asarray(ffwiz_df_pos_rank.loc[:, 'FFWIZ_Pos_Rank'])
    pos_diff = pd.DataFrame(adp_pos_rank_array - ffwiz_pos_rank_array, columns=["Pos_DIFF"])
    ffwiz_df_pos = pd.concat([ffwiz_df_pos_rank, adp_pos_rank, pos_diff, ffwiz_df_no_pos_rank], axis=1)

    return ffwiz_df_pos

### Generate Overall and Positional Rankings (Fxns of Fxns)

In [29]:
def generate_ffwiz_data(teams, ppr, put_fa_on_team, injured_players, first_playoff_week, last_playoff_week, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te, playoff_weight, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster, mccaffrey_penalty, barkley_penalty, najee_penalty):
    
    # pull in adp data and schedule based on PPR selection and # of teams in league (FROM USER)
    df_ffc = pull_adp(ppr, teams)
    
    # put free agents on teams or drop them
    df_mod_fa = modify_fa(df_ffc, put_fa_on_team)
    
    # remove duplicate players
    df_drop_duplicates = remove_duplicates(df_mod_fa)
    
    # drop injured players
    df = drop_injured_players(df_drop_duplicates, injured_players)
    
    print(len(df))
    print(len(df_ffc))
    
    #load nfl schedule
    nfl_sched = load_schedule()

    # make dataframe with each players opponents
    opponents_df = sched_per_player(df, nfl_sched)

    # calculate defense and offense strength scores
    defense_score = calc_defense_strength(bye_adjustor)
    offense_score = calc_offense_strength(bye_adjustor)

    # map matchup strengths for each player for every week
    str_scores_unwgt = weekly_matchup_strength(df, opponents_df, offense_score, defense_score)

    # generate weekly weights based on inputted playoff weeks (FROM USER)
    week_weights_qb_wr_k_def, week_weights_rb_te = week_weight_table(first_playoff_week, last_playoff_week, week_weights_exponent, playoff_weight, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te)

    # weight each weekly matchup for each player
    str_scores_wgt = weight_str_scores(df, str_scores_unwgt, week_weights_qb_wr_k_def, week_weights_rb_te, first_playoff_week, last_playoff_week, playoff_weight)

    # get ONE value per player by summing all weighted weekly matchup scores and SCALING so mean=0, var=1
    str_scores_scaled = player_weighted_sched_str(str_scores_wgt)
    
    # get ffwiz absolute score rankings based on further adjusting of differentials
    ffwiz_score = ffwiz_score_scaled(mccaffrey_penalty, barkley_penalty, najee_penalty, df, str_scores_scaled, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster)

    # get overall ffwiz rankings
    ffwiz_df_overall = ffwiz_overall_rankings(df, ffwiz_score)

    # get positional ffwiz rankings
    ffwiz_df_pos = ffwiz_df_positional(ffwiz_df_overall)
    
    return ffwiz_df_overall, ffwiz_df_pos

### Make List of Dataframe Outputs

In [30]:
def make_dataframes_lists(teams_array, ppr_array, put_fa_on_team, injured_players, first_playoff_week_array, last_playoff_week_array, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te, playoff_weight, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster, mccaffrey_penalty, barkley_penalty, najee_penalty):
    ffwiz_overall_list = []
    ffwiz_pos_list = []
    tab_name_list = []

    # iterate through all user inputs
    for i in range(0, len(teams_array)):
        for j in range(0, len(ppr_array)):
            for k in range(0, len(first_playoff_week_array)):
                for l in range(0, len(last_playoff_week_array)):
                    
                    #generate rankings
                    ffwiz_df_overall, ffwiz_df_pos = generate_ffwiz_data(teams_array[i], ppr_array[j], put_fa_on_team, injured_players, first_playoff_week_array[k], last_playoff_week_array[l], week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te, playoff_weight, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster, mccaffrey_penalty, barkley_penalty, najee_penalty)

                    ffwiz_overall_list.append(ffwiz_df_overall)
                    ffwiz_pos_list.append(ffwiz_df_pos)
                    
                    #make list of tabnames for excel file
                    tab_name =  teams_array[i] + '_' + str(j) + '_' + str(first_playoff_week_array[k]) + '_' + str(last_playoff_week_array[l])
                    tab_name_list.append(tab_name)
    
                    # print to keep track of progress
                    print(tab_name)
                    print(str(datetime.now()))
    
    return ffwiz_overall_list, ffwiz_pos_list, tab_name_list       

### Make Excel File with Overall and Positional Rankings Tabs

In [31]:
def make_excel(ffwiz_overall_list, ffwiz_pos_list, tab_name_list):

    # make writer object for Excel file
    writer = pd.ExcelWriter('./rankings_data_files/rankings_data_master_2021.xlsx', engine='xlsxwriter')
    
    # iterate through every input permutation
    for i in range(0, len(tab_name_list)):
        
        # generate breakpoints of positions in order to split tabs by position
        idx_list = []
        for j in range(0, len(ffwiz_pos_list[i])):
                if ffwiz_pos_list[i].loc[j, 'FFWIZ_Pos_Rank'] == 1:
                    idx_list.append(j)
        idx_list.append(j) #add END value for indexing

        # create positional dataframes based on breakpoints above
        ffwiz_def = ffwiz_pos_list[i].iloc[idx_list[0]:idx_list[1]]
        ffwiz_k = ffwiz_pos_list[i].iloc[idx_list[1]:idx_list[2]]
        ffwiz_qb = ffwiz_pos_list[i].iloc[idx_list[2]:idx_list[3]]
        ffwiz_rb = ffwiz_pos_list[i].iloc[idx_list[3]:idx_list[4]]
        ffwiz_te = ffwiz_pos_list[i].iloc[idx_list[4]:idx_list[5]]
        ffwiz_wr = ffwiz_pos_list[i].iloc[idx_list[5]:idx_list[6]]

        # create tabs for overall and each position in Excel file
        ffwiz_overall_list[i].to_excel(writer, sheet_name=tab_name_list[i]+'_ALL')
        ffwiz_def.to_excel(writer, sheet_name=tab_name_list[i]+'_DEF')
        ffwiz_k.to_excel(writer, sheet_name=tab_name_list[i]+'_K')
        ffwiz_qb.to_excel(writer, sheet_name=tab_name_list[i]+'_QB')
        ffwiz_rb.to_excel(writer, sheet_name=tab_name_list[i]+'_RB')
        ffwiz_te.to_excel(writer, sheet_name=tab_name_list[i]+'_TE')
        ffwiz_wr.to_excel(writer, sheet_name=tab_name_list[i]+'_WR')
    
    writer.save()

### Inputs

In [32]:
# # GLOBAL INPUTS (constant regardless of user input)
week_weights_thru_wk14_qb_wr_k_def = [1,0.99,0.98,0.97,0.95,0.92,0.88,0.83,0.77,0.65,0.54,0.45,0.45,0.45]  #based on games started analysis & trade assumptions
week_weights_thru_wk14_rb_te = [1,0.99,0.98,0.97,0.95,0.92,0.88,0.83,0.77,0.65,0.54,0.45,0.45,0.45]  #based on games started analysis & trade assumptions

k_rank_penalty = 4

def_rank_penalty = 1.5
wr_rank_booster = 0.0
bye_adjustor = -1.5

put_fa_on_team = pd.DataFrame([["Alvin Kamara", "NO"],["Saquon Barkley", "NYG"]])
injured_players = ["Andrew Luck"]

# # USER INPUTS
# teams_array = ['8', '10', '12', '14']
# ppr_array = ['standard', 'ppr']
# first_playoff_week_array = [14, 15]
# last_playoff_week_array = [15, 16, 17]

# USER INPUTS
teams_array = ['10', '12']
ppr_array = ['standard', 'ppr']
first_playoff_week_array = [15, 16]
last_playoff_week_array = [16, 17]

# ADJUST TO SKEW RANKINGS
mccaffrey_penalty = 1.0
barkley_penalty = 1.0
najee_penalty = 0.1
rank_adder = 105    # lower number FAVORS ADP
matchup_strength_scaler = 30  # lower number FAVORS STRENGTH OF SCHEDULE
week_weights_exponent = 6    # higher number FAVORS EARLY SEASON GAMES
playoff_weight = 1  # higher number FAVORS PLAYOFF WEEKS

### Generate Rankings and Make Excel Output (Run Functions!)

In [33]:
ffwiz_overall_list, ffwiz_pos_list, tab_name_list = make_dataframes_lists(teams_array, ppr_array, put_fa_on_team, injured_players, first_playoff_week_array, last_playoff_week_array, week_weights_thru_wk14_qb_wr_k_def, week_weights_thru_wk14_rb_te, playoff_weight, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty, wr_rank_booster, mccaffrey_penalty, barkley_penalty, najee_penalty)

https://fantasyfootballcalculator.com/adp/csv/standard.csv?teams=10&position=all
223
223
[1.0, 0.941480149401, 0.8858423808639999, 0.8329720049289998, 0.7350918906249998, 0.6063550013440001, 0.464404086784, 0.3269403733689999, 0.20842238008900002, 0.07541889062500001, 0.02479491129600001, 0.008303765625, 0.008303765625, 0.008303765625, 1, 1, 0.0, 0.0]
    FFWIZ  ADP  DIFF  Rank                   Name Pos Team Bye
0       1    1     0   1.3    Christian McCaffrey  RB  CAR  13
1       2    2     0   2.6            Dalvin Cook  RB  MIN   7
2       3    6     3   6.1             Nick Chubb  RB  CLE  13
3       4    3    -1   3.0          Derrick Henry  RB  TEN  13
4       5    5     0   5.1        Ezekiel Elliott  RB  DAL   7
5       6    9     3   9.1         Saquon Barkley  RB  NYG  10
6       7   13     6  12.3           Najee Harris  RB  PIT   7
7       8    4    -4   4.2           Alvin Kamara  RB   NO   6
8       9    7    -2   7.3            Aaron Jones  RB   GB  13
9      10   15  

https://fantasyfootballcalculator.com/adp/csv/standard.csv?teams=10&position=all
223
223
[1.0, 0.941480149401, 0.8858423808639999, 0.8329720049289998, 0.7350918906249998, 0.6063550013440001, 0.464404086784, 0.3269403733689999, 0.20842238008900002, 0.07541889062500001, 0.02479491129600001, 0.008303765625, 0.008303765625, 0.008303765625, 0.45, 1, 1, 0.0]
    FFWIZ  ADP  DIFF  Rank                   Name Pos Team Bye
0       1    1     0   1.3    Christian McCaffrey  RB  CAR  13
1       2    2     0   2.6            Dalvin Cook  RB  MIN   7
2       3    6     3   6.1             Nick Chubb  RB  CLE  13
3       4    3    -1   3.0          Derrick Henry  RB  TEN  13
4       5    4    -1   4.2           Alvin Kamara  RB   NO   6
5       6   10     4   9.4        Jonathan Taylor  RB  IND  14
6       7    5    -2   5.1        Ezekiel Elliott  RB  DAL   7
7       8   15     7  14.1         Antonio Gibson  RB  WAS   9
8       9    9     0   9.1         Saquon Barkley  RB  NYG  10
9      10   13 

https://fantasyfootballcalculator.com/adp/csv/ppr.csv?teams=10&position=all
202
202
[1.0, 0.941480149401, 0.8858423808639999, 0.8329720049289998, 0.7350918906249998, 0.6063550013440001, 0.464404086784, 0.3269403733689999, 0.20842238008900002, 0.07541889062500001, 0.02479491129600001, 0.008303765625, 0.008303765625, 0.008303765625, 0.45, 1, 0.0, 0.0]
    FFWIZ  ADP  DIFF  Rank                   Name Pos Team Bye
0       1    1     0   1.2    Christian McCaffrey  RB  CAR  13
1       2    2     0   2.4            Dalvin Cook  RB  MIN   7
2       3    4     1   3.8          Derrick Henry  RB  TEN  13
3       4   10     6   9.5             Nick Chubb  RB  CLE  13
4       5    3    -2   3.7           Alvin Kamara  RB   NO   6
5       6    5    -1   5.4        Ezekiel Elliott  RB  DAL   7
6       7    6    -1   6.5          Davante Adams  WR   GB  13
7       8   13     5  12.4           Najee Harris  RB  PIT   7
8       9    7    -2   7.3            Aaron Jones  RB   GB  13
9      10    9    

https://fantasyfootballcalculator.com/adp/csv/standard.csv?teams=12&position=all
223
223
[1.0, 0.941480149401, 0.8858423808639999, 0.8329720049289998, 0.7350918906249998, 0.6063550013440001, 0.464404086784, 0.3269403733689999, 0.20842238008900002, 0.07541889062500001, 0.02479491129600001, 0.008303765625, 0.008303765625, 0.008303765625, 1, 1, 1, 0.0]
    FFWIZ  ADP  DIFF  Rank                   Name Pos Team Bye
0       1    1     0   1.3    Christian McCaffrey  RB  CAR  13
1       2    2     0   2.6            Dalvin Cook  RB  MIN   7
2       3    6     3   6.1             Nick Chubb  RB  CLE  13
3       4    3    -1   3.0          Derrick Henry  RB  TEN  13
4       5    5     0   5.1        Ezekiel Elliott  RB  DAL   7
5       6    4    -2   4.2           Alvin Kamara  RB   NO   6
6       7    9     2   9.1         Saquon Barkley  RB  NYG  10
7       8   15     7  14.1         Antonio Gibson  RB  WAS   9
8       9   13     4  12.3           Najee Harris  RB  PIT   7
9      10   10    

https://fantasyfootballcalculator.com/adp/csv/ppr.csv?teams=12&position=all
202
202
[1.0, 0.941480149401, 0.8858423808639999, 0.8329720049289998, 0.7350918906249998, 0.6063550013440001, 0.464404086784, 0.3269403733689999, 0.20842238008900002, 0.07541889062500001, 0.02479491129600001, 0.008303765625, 0.008303765625, 0.008303765625, 1, 1, 0.0, 0.0]
    FFWIZ  ADP  DIFF  Rank                   Name Pos Team Bye
0       1    1     0   1.2    Christian McCaffrey  RB  CAR  13
1       2    2     0   2.4            Dalvin Cook  RB  MIN   7
2       3   10     7   9.5             Nick Chubb  RB  CLE  13
3       4    4     0   3.8          Derrick Henry  RB  TEN  13
4       5    5     0   5.4        Ezekiel Elliott  RB  DAL   7
5       6   13     7  12.4           Najee Harris  RB  PIT   7
6       7    3    -4   3.7           Alvin Kamara  RB   NO   6
7       8    6    -2   6.5          Davante Adams  WR   GB  13
8       9    9     0   9.5          Austin Ekeler  RB  LAC   7
9      10   11     1 

https://fantasyfootballcalculator.com/adp/csv/ppr.csv?teams=12&position=all
202
202
[1.0, 0.941480149401, 0.8858423808639999, 0.8329720049289998, 0.7350918906249998, 0.6063550013440001, 0.464404086784, 0.3269403733689999, 0.20842238008900002, 0.07541889062500001, 0.02479491129600001, 0.008303765625, 0.008303765625, 0.008303765625, 0.45, 1, 1, 0.0]
    FFWIZ  ADP  DIFF  Rank                   Name Pos Team Bye
0       1    1     0   1.2    Christian McCaffrey  RB  CAR  13
1       2    2     0   2.4            Dalvin Cook  RB  MIN   7
2       3    4     1   3.8          Derrick Henry  RB  TEN  13
3       4    3    -1   3.7           Alvin Kamara  RB   NO   6
4       5   10     5   9.5             Nick Chubb  RB  CLE  13
5       6    5    -1   5.4        Ezekiel Elliott  RB  DAL   7
6       7    6    -1   6.5          Davante Adams  WR   GB  13
7       8    7    -1   7.3            Aaron Jones  RB   GB  13
8       9   13     4  12.4           Najee Harris  RB  PIT   7
9      10    9    -1

In [34]:
make_excel(ffwiz_overall_list, ffwiz_pos_list, tab_name_list)